# ⚛️ H₂ VQE — Fermion-to-Qubit Mapping Comparison (Noiseless)

This notebook compares **fermion-to-qubit mappings** for the hydrogen molecule **H₂**
using a fixed VQE setup and your packaged VQE engine.

We vary only the **mapping**:

- `jordan_wigner`
- `bravyi_kitaev`
- `parity`

Everything else is held constant:

- Molecule: **H₂** (equilibrium geometry, STO-3G)
- Ansatz: **UCCSD** (singles + doubles)
- Optimizer: **Adam**
- Device: `default.qubit` (noiseless)
- Steps: 60

---

## 🎯 Goals

For each mapping, we obtain:

1. **Hamiltonian complexity** (number of Pauli terms)
2. **VQE convergence trace** (energy vs iteration)
3. **Final ground-state energy**

This helps illustrate how different encodings affect:

- Hamiltonian length (Pauli term count)
- Optimisation behaviour
- Resource requirements

while keeping the underlying chemistry and ansatz fixed.

---

Internally, we use the high-level API:

```python
from vqe.core import run_vqe_mapping_comparison
```

which:

- Builds the qubit Hamiltonian for each mapping
- Calls `run_vqe` for each case
- Reuses the caching & plotting infrastructure
- Returns a summary dictionary for downstream analysis


## 🔧 Imports & Configuration

In [ ]:
import sys
import os

import matplotlib.pyplot as plt
from pennylane import numpy as np

# Add project root to path (for direct notebook use)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe_mapping_comparison
from vqe.io_utils import IMG_DIR

mappings = ["jordan_wigner", "bravyi_kitaev", "parity"]

print("Mappings to compare:", mappings)
print("Images will be saved under:", IMG_DIR)

## 🚀 Run VQE Mapping Comparison (Package API)

We now call:

```python
run_vqe_mapping_comparison(
    molecule="H2",
    ansatz_name="UCCSD",
    optimizer_name="Adam",
    mappings=mappings,
    steps=60,
    stepsize=0.2,
    noisy=False,
)
```

This function:

- Builds the molecular Hamiltonian for each mapping
- Runs VQE with the chosen ansatz and optimizer
- Produces a convergence plot (energy vs iteration) saved to `IMG_DIR`
- Returns, for each mapping:

```python
{
    "final_energy": float,
    "energies": [...],
    "num_qubits": int,
    "num_terms": int or None,
}
```

In [ ]:
results = run_vqe_mapping_comparison(
    molecule="H2",
    ansatz_name="UCCSD",
    optimizer_name="Adam",
    mappings=mappings,
    steps=60,
    stepsize=0.2,
    noisy=False,
    force=False,
    show=True,
)

results

## 📊 Summary Table — Final Energies & Hamiltonian Sizes

In [ ]:
print("\nH₂ VQE — Mapping Comparison Summary (UCCSD, noiseless)\n")

header = f"{'Mapping':>15}  {'Qubits':>6}  {'Pauli terms':>12}  {'E_final (Ha)':>14}"
print(header)
print("-" * len(header))

for mapping in mappings:
    data = results[mapping]
    n_qubits = data["num_qubits"]
    n_terms = data["num_terms"]
    E_final = data["final_energy"]

    n_terms_str = str(n_terms) if n_terms is not None else "N/A"
    print(
        f"{mapping:>15}  "
        f"{n_qubits:6d}  "
        f"{n_terms_str:>12}  "
        f"{E_final:14.8f}"
    )

## 🧾 Summary

In this notebook you:

- Ran a **noiseless VQE** simulation for H₂ with a **UCCSD ansatz**
- Compared **fermion-to-qubit mappings**:
  - Jordan–Wigner
  - Bravyi–Kitaev
  - Parity
- Inspected:
  - Final ground-state energies for each mapping
  - Convergence behaviour (energy vs iteration) from package-generated plots

This provides a compact, **production-style** demonstration of how the choice
of encoding affects both **resources** and **optimisation** for a simple
benchmark molecule.